This notebook is where we are developing the code for the decision tree.

`hopkins-knowledge.csv` contains the knowledge base from [here](https://github.com/drdevinhopkins/20_Questions/blob/master/knowledge_base.csv); used only for development.

**HOW TO PLAY: Enter 0 for no and 1 for yes.**

In [2]:
import pandas as pd
import numpy as np

In [3]:
kn = pd.read_csv('hopkins-knowledge.csv')

y = kn['Animal']
X = kn.loc[:, 'Hair':'Invertebrate']
print('There are {0} objects and {1} features for each object.'.format(y.shape[0], X.shape[1]))

There are 100 objects and 28 features for each object.


### TODO for Anna:

In [4]:
def ask_about_feature(feat_name, counter):
    """
    ANNA: Modify this function to print out a natural language question based on the feature name,
    e.g. "Does it have wings?"
    
    Arg:
        feat_name: string, name of feature to split dataset on
        counter: int, the count of the current question.
    Prints:
        A string, the natural language question asking about that feature.
    Returns:
        Nothing.
    """
    question = feat_name+'?'
    print('Q'+str(counter)+': '+question)


def ask_about_object(obj_name, counter):
    """
    ANNA: Modify this function to print out a natural language question based on the object name, 
    e.g. "Are you thinking of an ocelot?"
    
    Arg:
        obj_name: string, name of object to guess.
        counter: int, the count of the current question.
    Prints:
        A string, the natural language question guessing that object.
    Returns:
        Nothing.
    """
    question = obj_name+'?'
    print('Q'+str(counter)+': '+question)
    

### TODO for Rodrigo:

- I only made one `endgame_lose()` function at all of the points in the decision tree where the game is lost. You might want to split this up into something like `endgame_lose_questionlimit()` and `endgame_lose_guessedwrong()` etc., since that might make it more clear what action needs to be taken

In [5]:
def endgame_lose(answers):
    """
    RODRIGO: If the game is lost, we will need to figure out why (was the 20Q limit reached? Or was the user's 
    object not in the knowledge base?) and take action based on that. The code to add in unknown objects can
    be incorporated here.
    I've set the code up so that the answers dictionary ends up here, hopefully that makes your life easier.
    
    Arg:
        answers: a dictionary where the keys are features and the values are the user's answers to those features
    """
    #======================================
    
    #Giving access to global variables
    global kn
    
    #======================================
    
    #Swallowing pride
    print('dangit, you were too smart for me!')
    
    #======================================
    
    #Getting correct answer
    print('Which object were you thinking about? (please_write_it_in_this_format)')
    correct_answer = input().lower()
    #adding the correct answer to the answers dict
    answers[y.name] = correct_answer
    print(answers)
    print('Smart choice!')
    
    #=======================================
    
    #Asking for more info
    
    #=======================================
    
    # If the correct answer is already in our dataset
    if correct_answer in y.unique():
        # If the user's answers contradict our KB we will add a new row to the KB with the new information
        
        #temporary array to keep the updated row
        correct_answer_index = np.where(y==correct_answer)[0][0]
        new_row = kn.iloc[[correct_answer_index]].copy()
        
        #update process
        for attribute, value in answers.items():
            if type(value) != str: #making sure to not compare the animal name
                if (value == new_row[attribute]).bool() == False: #diff than in our KB
                            new_row[correct_answer_index, attribute] = value
        kn = kn.append(new_row, ignore_index=True)
     
    #if correct answer is not yet in our dataset
    else: 
        #blank new row
        new_row = []
        
        #filling in the new row
        for i, attribute in enumerate(kn.columns, 0):
            if attribute in answers.keys(): #knowledge provided by the user
                new_row.append(answers[attribute])
            else:
                new_row.append(993993)
        
        #adding it to the KB
        final = dict()
        for i, at in enumerate(kn.columns, 0):
            final[at] = new_row[i]
        kn = kn.append(final, ignore_index=True)
        
        
def endgame_win():
    """
    RODRIGO: Does something in the event that the game was won.
    """
    print('oh yeah! I rock')

Done by Elizabeth:

In [ ]:
def dist_from_1(feat_col):
    """
    Returns the absolute distance from 1 of the split cardinality ratio for the given column of X.
    """
    counts = feat_col.value_counts()
    if len(counts) == 2:  # i.e. if there are both 1s and 0s in the column
        ratio = counts[0] / counts[1] 
        return abs( 1 - ratio )
    return np.nan  # Features that get NaNs are filtered out later :)


def rank_features(df):
    """
    Ranks all features in df by increasing absolute distance from 1 of the SCR.
    """
    return df.apply(dist_from_1).sort_values()


# The trick now will be to turn this ranking into a probability distribution.
# We can't just normalise to convert this into one, because the best choices have the smallest values, so they'd be least likely to be selected.
# One way might be to find the max value (that's not the flag 10e10) and subtract each $abs(1-SCR)$ from that and then normalise, and assign the ones with value 10e10 a prob of zero.
# Let's see if that works.


def get_distinguishing_feats(X):
    """
    Ranks the features in X in ascending order of abs(1-SCR) and filters out those that contain either all 0s or all 1s
    (i.e. those that cannot be used to distinguish between objects).
    
    Arg:
        X: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
    Returns:
        A pandas series of features ranked by abs(1-SCR) ascending, with non-distinguishing features removed.
    """
    # Rank the features, drop the NaNs that were put there by dist_from_1(), and return what remains.
    r = rank_features(X)
    distinguishing_feats = r.dropna()
    return distinguishing_feats


def sample_feature(X, distinguishing_feats):
    """
    Ranks the features in X, creates a probability distribution from the ranking, and samples a feature
    according to this probability distribution, returning this as the feature to ask about. Also returns
    the number of distinguishing features (i.e. those that contain both 0 and 1), as a check for when to 
    stop trying to split on features.
    
    Arg:
        X: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
        distinguishing_feats: pandas series of features ranked by abs(1-SCR) ascending, with non-distinguishing 
                              features removed.
    Returns:
        sampled_feat: A string, the sampled feature to ask about.
    """

    # Get the max value of the distinguishing features (this is the final element, since they're ranked ascending).
    max_val = distinguishing_feats[-1]

    # Subtract each value in the series from max_val+1; now the features will be sorted descending, and the best features
    # to split on will have the highest values.
    # (the +1 is there because otherwise the final feature will have probability 0, and we still want it to be eligible,
    # if improbable)
    ranked_feats_transf = max_val - distinguishing_feats + 1 

    # Convert to a probability distribution by dividing by the sum of all observations.
    feat_prob_dist = ranked_feats_transf / ranked_feats_transf.sum()

    # Sample one feature from this distribution and return that feature.
    sampled_feat = np.random.choice( feat_prob_dist.index, 1, p = feat_prob_dist )
    sampled_feat = str(sampled_feat[0])
    
    return sampled_feat


def split_df_on_feature(df, feature, answer):
    """
    Returns subset of df where df[feature]==answer and drops feature from columns in df.
    
    Args:
        df: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
        feature: string, the column name to split on
        answer: int, 0 or 1, reflecting which subset of the dataframe to keep
    Returns:
        pandas dataframe with features as columns (subset of df).
    """
    return df[df[feature] == answer].drop(columns=[feature])



def ask_and_process_answer(feature, counter, df, answers):
    """
    Prints question about the supplied feature and gets the answer.
    If the answer is 1 or 0 (yes or no), splits the dataset, returning
    only those instances where the answer holds.
    If the answer is 2 (unknown), does not split the dataset, but removes
    that feature from it.
    Also adds the feature and answer to the answers dictionary.
    
    Args:
        feature: a string, a column in df
        counter: int, the count of the current question.
        df: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
        answers: a dictionary where the keys are features and the values are the user's answers to those features
    Returns:
        pandas dataframe with features as columns.
        answers dict with feature added.
    """
    ask_about_feature(feature, counter)
    answ = int( input() )
    
    # Check for bad input.
    while answ not in set([0, 1, 2]):
        print('Please give valid input (0=no, 1=yes, 2=unknown).')
        answ = int( input() )
    
    # Add answer to the answers database (even if it's unknown... Rodrigo can choose how to deal with those)
    answers[feature] = answ
    
    # Drop the feature from the dataset without splitting if the answer is 2; else, split on feature.
    if answ == 2:
        return df.drop(columns=[feature]), answers
    else:
        return split_df_on_feature(df, feature, answ), answers
    
    
def guess_rem_objs(y, X_idcs, counter, answers):
    """
    If dataset cannot be split by features anymore, but multiple objects still remain, this function goes through
    them in a random order, guessing each in a randomised order until endgame.
    
    Args:
        y: pandas series, all objects in dataset
        X_idcs: 'pandas.core.indexes.numeric.Int64Index', the remaining indices to choose from (all non-candidates 
                having been pruned)
        counter: int, the count of the current question.
        answers: a dictionary where the keys are features and the values are the user's answers to those features
    Returns:
        Nothing.
    """
   
    # Subset the ys based on X_idcs and shuffle them, so the guessing will happen in a random order.
    ys_to_guess = y[y.index.isin(X_idcs)]
    ys_to_guess = ys_to_guess.sample(frac=1, random_state=3)

    # Go through ys_to_guess and ask about each object, checking the counter each time to make sure we don't exceed
    # 20 questions here either. If guessed correctly, enter endgame_win and stop looping.
    for guess in ys_to_guess:
        if counter <= 20:
            ask_about_object(guess, counter)
            counter += 1
            answ = int( input() )
            if answ == 1:
                endgame_win()
                break
    
    # If we made it out of the for loop and the final answer isn't 1, that means we lost.
    if not answ:
        endgame_lose(answers)
#         quick_endgame_lose()
        return


def quick_endgame_lose():
    print('dangit')

In [7]:
# Stuff added by Elizabeth
# Beginning to look at how to maintain probability distribution over animals 
# TODO: incorporate it into play().
# Sampling features will still work on the principle of iterative database bisection, but we won't take the candidate animals
# based on what remains in that pared-down dataset anymore.
# Instead, we will guess them in decreasing order of probability based on answers given by the user.

def init_animal_probdist(y):
    """
    Initialises the prior probability distribution with a uniform prior of 20 across all animals.
    
    Arg:
        y: pandas series, all objects in dataset
    Returns:
        A series with the animals as index and their uniform prior probability as value
    """
    # Convert y into a dataframe (with one column: Animal), add column for probability, and turn Animal col into index
    # and save again as series (will be easier to deal with later).
    y_df = pd.DataFrame(y)
    y_df['prob'] = np.repeat(20, len(y))
    y_df = y_df.set_index('Animal')['prob']
    
    # Note to self: Get list of animals with y_df.index, and get probability of animal by e.g. y_df.loc['antelope'].
    return y_df
    

def update_animal_probdist(y_probdist, kn, feature_asked, answ):
    """
    Given a user's answers to a question about a particular feature, updates the probability distribution over animals.
    
    Args:
        y_probdist: pandas series with the animals as index and their uniform prior probability as value
        kn: the complete knowledge base
        feature_asked: a string, the feature just asked about
        answ: an integer, the user's response
    Returns:
        Pandas dataframe, y_probdist updated to reflect the answer provided by the user 
    """
    
    # Set the index of kn to the Animal column for easy combination with the probability distribution.
    kn = kn.set_index('Animal')
    
    # Extract the column in kn corresponding to the feature we asked about.
    kn_col = pd.Series(kn[feature_asked])  # Copy this column before modifying it, so that we don't modify kn!
            
    # If the user's answer is feature_asked == 1, then all animals with feature_asked == 0 get -1 in y_probdist.
    # To accomplish this, subtract 1 from kn_col to take 1 to 0 and 0 to -1, and then add that to y_probdist.
    if answ == 1:
        kn_col -= 1
        return y_probdist + kn_col
    
    # If the user's answer is feature_asked == 0, then all animals with feature_asked == 1 get -1 in y_probdist.
    # To accomplish this, subtract k_col from y_probdist.
    elif answ == 0:
        return y_probdist - kn_col
    
test_probdist = init_animal_probdist(y)
update_animal_probdist(test_probdist, kn, 'Hair', 1)    

Animal
aardvark    20
antelope    20
bass        19
bear        20
boar        20
            ..
wallaby     20
wasp        20
wolf        20
worm        19
wren        19
Length: 100, dtype: int64

In [8]:
def play(kn, X, y, y_probdist, counter, answers):
    """
    Recursively bisects knowledge base based on user input about whether target object matches the feature.
    
    Args:
        X: pandas dataframe with features as columns, populated by 0s and 1s, one row per instance
        y: pandas series, all objects in dataset
        y_probdist: pandas series with the animals as index and their uniform prior probability as value
        counter: int, the count of the current question.
        answers: a dictionary where the keys are features and the values are the user's answers to those features
    Returns:
        nothing.
    """
    
    # =============================
    # BASE CASE 0: counter > 20
    # =============================
    if counter > 20:
    #         print('TOO MANY QUESTIONS!')
        endgame_lose(answers)
        return

    # =============================
    # BASE CASE 1: Only one row left in the data, so only one object available to guess.
    # =============================

    if len(X) == 1:
    #         print('ONLY ONE OBJECT LEFT!')
        guess = y[X.index].to_string(index=False)  # (all this machinery required to print pd.Series as str, sigh)
        ask_about_object(guess, counter)
        answ = int( input() )
        endgame_win() if answ == 1 else endgame_lose(answers)
        return

    # =============================
    # BASE CASE 2: Only one feature left in the data (have asked about all other ones). Will need to ask about that feature, 
    # subset the data correspondingly, and then go through all remaining objects.
    # =============================

    if len(X.columns) == 1:
    #         print('ONLY ONE FEATURE LEFT!')
        feature_to_split_on = X.columns[0]
        X_bc2, answers = ask_and_process_answer(feature_to_split_on, counter, X, answers)
        counter += 1

        # If there are no remaining objects to guess after splitting the data on this feature, then endgame_lose().
        if len(X_bc2.index) == 0:
    #             print('NO OBJECTS LEFT TO GUESS!')
            endgame_lose(answers)
            return
        # Otherwise, cycle through all remaining objects until endgame.
        else:
            guess_rem_objs(y, X_bc2.index, counter, answers)  # includes endgame
            return

    # =============================
    # BASE CASE 3: There are no more distinguishing features, so the dataset can't be divided anymore. 
    # Will just need to cycle through all remaining objects until endgame.
    # =============================

    disting_feats = get_distinguishing_feats(X)

    # Count the distinguishing features in X (i.e. those that aren't all 0s or all 1s) and cycle through objects 
    # if there are none.
    if len( disting_feats ) == 0:
    #         print('NO MORE DISTINGUISHING FEATURES!')
        guess_rem_objs(y, X.index, counter, answers)  # includes endgame
        return


    # =============================
    # RECURSIVE CASE: If we get this far, that means we didn't fall into any of the base cases, so the game can be played!
    # =============================

    #     print(disting_feats, '\n') # Sanity check, can rm this later
    #     print(X, '\n')

    # Sample a feature disting_feats proportional to how well it splits the data in X and ask about it.
    feat_to_split_on = sample_feature(X, disting_feats)
    X_new, answers = ask_and_process_answer(feat_to_split_on, counter, X, answers)
    counter += 1
    play(kn, X_new, y, y_probdist, counter, answers)

In [10]:
y_probdist = init_animal_probdist(y)

# play(kn, X, y, y_probdist, counter=1, answers=dict())

Q1: Eggs?
1
Q2: Toothed?
1
Q3: Predator?
1
Q4: Catsize?
0
Q5: Fish?
0
Q6: Tail?
1
Q7: Nlegs_4?
0
Q8: Venomous?
1
Q9:  pitviper?
1
oh yeah! I rock
